In [1]:
import psycopg2
import json
import pandas as pd
import time
from psycopg2.extras import execute_batch

In [2]:
start_time = time.time()

In [3]:
DB_NAME = "next_gen"
DB_USER = "sde"
DB_PASS = "sde"
DB_HOST = "salesiqgen2.cygagau4oro0.us-west-2.rds.amazonaws.com"
DB_PORT = "5432"

In [4]:
try:
    conn = psycopg2.connect(database=DB_NAME,
                            user=DB_USER,
                            password=DB_PASS,
                            host=DB_HOST,
                            port=DB_PORT)
    print("Database connected successfully")
except Exception as e:
    print(f"Database not connected successfully: {e}")

Database connected successfully


In [5]:
cur = conn.cursor()

In [6]:
def find_max_level(cur):
    max_level = 0
    level = 1
    while True:
        code_column = f"level{level}_code"
        cur.execute(f"SELECT COUNT(*) FROM qa_mergetest5_position_geo_temp WHERE {code_column} IS NOT NULL")
        count = cur.fetchone()[0]
        if count == 0:
            break
        max_level = level
        level += 1
    return max_level

max_level = find_max_level(cur)

In [7]:
# Fetch distinct metric keys present in the JSON data and store them in a list
cur.execute("""
    SELECT DISTINCT jsonb_object_keys(CAST(metric_data AS jsonb)) AS metric_key
    FROM qa_mergetest5_position_geo_temp
""")
metric_keys = [row[0] for row in cur.fetchall()]

# Prepare metric data items for the SQL query
metric_data_items = ', '.join([f"'{metric_key}', SUM(COALESCE((metric_data::jsonb->>'{metric_key}')::float, 0))" for metric_key in metric_keys])

In [8]:
try:
    query_parts = []
    for level in range(1, max_level + 1):
        code_column = f"level{level}_code"
        code_name = f"level{level}_name"
        query_parts.append(f"""
            SELECT 
                {code_column} AS code,
                {code_name} AS name,
                {level} AS level, 
                jsonb_build_object(
                    {metric_data_items}
                ) AS metric_data,
                ST_Union(shape) AS shape
            FROM qa_mergetest5_position_geo_temp
            GROUP BY {code_column}, {code_name}
        """)

    combined_query = " UNION ALL ".join(query_parts)

    cur.execute(combined_query)
    results = cur.fetchall()

    columns = ["code", "name", "level", "metric_data", "shape"]
    df = pd.DataFrame(results, columns=columns)

    df["metric_data"] = df["metric_data"].apply(lambda x: json.dumps(x))

    # Convert DataFrame to list of tuples
    records = df.to_records(index=False)
    records_list = [(r[0], r[1], int(r[2]), r[3], r[4]) for r in records]

    # Define the insert query
    insert_query = """
        INSERT INTO hg_positions (code, name, level, metric_data, shape)
        VALUES (%s, %s, %s, %s, %s)
    """

    # Bulk insert using execute_batch
    execute_batch(cur, insert_query, records_list)

    # Commit the transaction
    conn.commit()

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # Close the cursor and the connection
    cur.close()
    conn.close()

In [9]:
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Script completed in {elapsed_time:.2f} seconds")

Script completed in 45.83 seconds
